<a href="https://colab.research.google.com/github/dajinfx/hanghaeAI2/blob/main/W5_2_RagAdvance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##과제: 다양한 형태의 입력을 가지는 LLM 서비스 개발

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dajinfx/hanghaeAI2/blob/main/W5_2_RagAdvance.ipynb)

In [ ]:
!pip install langchain-community langchain-chroma langchain-openai bs4

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


In [ ]:
import openai
import json
import requests
import numpy as np
import torch
import pandas as pd
from langchain.prompts import PromptTemplate

In [ ]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.2 MB/s eta 0:00:00


In [ ]:
pip install unstructured

In [ ]:
pip install "unstructured[pdf]"

In [ ]:
gpt_model="gpt-4o"
temperature = 0.7  # 각 token을 샘플링할 때 사용하는 temperature 값입니다.
max_tokens = 4096  # 생성하는 최대 token 개수 입니다.
frequency_penalty = 0.0  # 같은 단어가 반복적으로 나오는 것을 방지하기 위한 옵션입니다.

##[MyCode] Upload apikey to googlelab

In [ ]:
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
    print(f" {filename} is uploaded")

with open('OpenAI_APIKey.txt', 'r') as file:
    api_key = file.read()

Saving OpenAI_APIKey.txt to OpenAI_APIKey.txt
 OpenAI_APIKey.txt is uploaded


In [ ]:
from langchain.document_loaders import PyPDFLoader
#from langchain.document_loaders import UnstructuredFileLoader

In [ ]:
file_url = "https://arxiv.org/pdf/2005.11401"

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

##[MyCode] UnstructuredURLLoader

UnstructuredURLLoader 로서 PDF 업로드 시도 실패했습니다,  원인 불명, 파악필요.

In [ ]:
'''
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 修复路径
from langchain.schema import Document

# 使用 UnstructuredURLLoader 加载 PDF 文件
loader = UnstructuredURLLoader(urls=[file_url])
docs = loader.load()
'''

ERROR:langchain_community.document_loaders.url:Error fetching or processing https://arxiv.org/pdf/2005.11401, exception: Unable to get page count. Is poppler installed and in PATH?


##[MyCode] PyPDFLoader

PyPDFLoader 이용으로  업로드성공

In [ ]:
loader = PyPDFLoader(
    file_path= file_url,
)
docs = loader.load()
print('논문 페이지 수:', len(docs))

논문 페이지 수: 19


In [ ]:
docs[0]

Document(metadata={'source': 'https://arxiv.org/pdf/2005.11401', 'page': 0}, page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡, Ethan Perez⋆,\nAleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,\nMike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†\n†Facebook AI Research; ‡University College London; ⋆New York University;\nplewis@fb.com\nAbstract\nLarge pre-trained language models have been shown to store factual knowledge\nin their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-\nstream NLP tasks. However, their ability to access and precisely manipulate knowl-\nedge is still limited, and hence on knowledge-intensive tasks, their performance\nlags behind task-speciﬁc architectures. Additionally, providing provenance for their\ndecisions and updating their world knowledge remain open research problems. Pre-\ntrained models with a differentiable access m

In [ ]:
# 打印加载的文档
for doc in docs:
    print(doc.page_content)

In [ ]:
# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,  # 각 청크의 최대 길이
    chunk_overlap=500  # 청크 간 겹침 부분 최소화
)

In [ ]:
#Chroma, Embedding
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=api_key)
)

In [ ]:
print(len(splits))

34


##LangChain Map-Reduce

In [ ]:
# Map에 사용할 prompt template
map_template = """다음은 문서 중 일부 내용입니다
{pages}
이 문서 목록을 기반으로 주요 내용을 요약해 주세요.

답변: """

# Map 프롬프트
map_prompt = PromptTemplate.from_template(map_template)

# Map에서 수행할 LLMChain 정의
llm = ChatOpenAI(temperature=0,
                 model_name=gpt_model,
                 api_key=api_key)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [ ]:
# Reduce 단계에서 처리할 프롬프트 정의
reduce_template = """다음은 요약의 집합입니다:
{doc_summaries}
이것들을 바탕으로 통합된 요약을 만들어 주세요.
요약을 할때,
1)***
2)***
3)***
이렇게 매개 요약마다 번호를 매겨주세요.
답변:"""

# Reduce 프롬프트 완성
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Reduce에서 수행할 LLMChain 정의
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import ReduceDocumentsChain

# 문서의 목록을 받아들여, 이를 단일 문자열로 결합하고, 이를 LLMChain에 전달합니다.
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="doc_summaries" # Reduce 프롬프트에 대입되는 변수
)

# Map 문서를 통합하고 순차적으로 Reduce합니다.
reduce_documents_chain = ReduceDocumentsChain(
    # 호출되는 최종 체인입니다.
    combine_documents_chain=combine_documents_chain,
    # 문서가 `StuffDocumentsChain`의 컨텍스트를 초과하는 경우
    collapse_documents_chain=combine_documents_chain,
    # 문서를 그룹화할 때의 토큰 최대 개수입니다.
    token_max=4000,
)

In [ ]:
from langchain.chains import MapReduceDocumentsChain

# 문서들에 체인을 매핑하여 결합하고, 그 다음 결과들을 결합합니다.
map_reduce_chain = MapReduceDocumentsChain(
    # Map 체인
    llm_chain=map_chain,
    # Reduce 체인
    reduce_documents_chain=reduce_documents_chain,
    # 문서를 넣을 llm_chain의 변수 이름(map_template 에 정의된 변수명)
    document_variable_name="pages",
    # 출력에서 매핑 단계의 결과를 반환합니다.
    return_intermediate_steps=False,
)

In [ ]:
# Map-Reduce 체인 실행
# 입력: 분할된 도큐먼트(②의 결과물)
result = map_reduce_chain.run(splits)
# 요약결과 출력
print(result)

1) ***RAG 모델의 성능과 특징***: RAG(회수-생성) 모델은 대규모 사전 학습된 언어 모델의 한계를 극복하기 위해 제안된 모델로, 파라메트릭 메모리와 비파라메트릭 메모리를 결합하여 지식 집약적인 작업에서 우수한 성능을 보입니다. RAG-Sequence와 RAG-Token의 두 가지 변형을 통해 다양한 NLP 작업에서 최첨단 성능을 달성하며, 특히 Jeopardy 질문 생성과 FEVER 사실 검증 작업에서 강력한 성능을 입증합니다. RAG 모델은 BART 모델에 비해 더 구체적이고 사실적으로 정확한 응답을 생성하며, 정보 검색과 생성 작업에서 더 효과적입니다.

2) ***NLP 작업에서의 검색 기법 활용과 사회적 영향***: 정보 검색은 개방형 질문 응답, 사실 확인, 장문 질문 응답 등 다양한 NLP 작업의 성능을 향상시킬 수 있으며, RAG 모델은 이러한 검색 기법을 활용하여 성능을 극대화합니다. RAG 모델은 Wikipedia와 같은 사실적 지식에 기반하여 더 정확하고 해석 가능한 결과를 생성할 수 있으며, 다양한 분야에서 긍정적인 사회적 이점을 제공합니다. 그러나 외부 지식 소스의 편견과 부정확성으로 인한 위험도 존재하며, 이를 완화하기 위한 AI 시스템의 활용이 필요합니다.

3) ***연구 동향과 모델 성능 비교***: NLP와 AI 분야의 최신 연구는 자연어 생성, 번역, 이해, 대화 시스템 등 다양한 응용 분야에서의 성능 향상과 문제 해결을 목표로 하고 있습니다. T5-11B 모델은 "closed-book" 오픈 도메인 QA 모델 중 가장 높은 성능을 보이며, RAG-Sequence 모델은 하이브리드 파라메트릭/비파라메트릭 접근을 통해 더 적은 파라미터로도 강력한 성능을 발휘합니다. 비파라메트릭 메모리 인덱스와 검색 컴포넌트의 문제점도 존재하지만, RAG 모델은 다양한 NLP 작업에서 기존의 최첨단 모델을 능가하는 성능을 보입니다.


##Conclusion

요약이 잘 출력이 되는걸로 보입니다.